In [3]:
import pydicom #read dicom files
import os
import pandas as pd
import shutil

In [15]:
labels_df = pd.read_excel("C:/Users/casti/Documents/Final_Project/lidc-idri nodule counts (6-23-2015).xlsx", index_col=0)
labels_df=labels_df.reset_index()
labels_df=labels_df[['TCIA Patent ID', 'Number of Nodules >=3mm**']]
labels_df.rename(columns = {'TCIA Patent ID':'ID', 'Number of Nodules >=3mm**':'severity_degree'}, inplace = True)
labels_df.set_index('ID')
labels_df['severity_degree'].replace(1, 0, inplace=True)
labels_df['severity_degree'].replace(2, 1, inplace=True)
labels_df['severity_degree'].replace(3, 1, inplace=True)
for i in range(4,30):
    labels_df['severity_degree'].replace(i, 2, inplace=True)
labels_df=labels_df.dropna()
labels_df2=labels_df.set_index('ID')
labels_df=labels_df2

In [16]:
labels_df

,severity_degree
ID,
LIDC-IDRI-0001,0
LIDC-IDRI-0002,0
LIDC-IDRI-0003,2
LIDC-IDRI-0004,0
LIDC-IDRI-0005,1
...,...
LIDC-IDRI-1008,2
LIDC-IDRI-1009,0
LIDC-IDRI-1010,0


In [17]:
data_dir = 'C:/Users/casti/Documents/Final_Project/LIDC-IDRI2/'

In [18]:
patients = os.listdir(data_dir)

In [19]:
patients

['LIDC-IDRI-0001', 'LIDC-IDRI-0002', 'LIDC-IDRI-0004', 'LIDC-IDRI-0005']

In [2]:
for patient in patients:
    data_dir2 = data_dir + patient
    fileammount=len(os.listdir(data_dir2))
    if fileammount<=10:
        shutil.rmtree(data_dir2)

NameError: name 'patients' is not defined

In [20]:
patients2=[]
patients3=[]
for patient in patients:
#    label = labels_df.get_value(patient, 'severity_degree')
    path = data_dir + patient
    fileammount=len(os.listdir(path))
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    patientID = slices[0].PatientID
    label = labels_df.get_value(patientID, 'severity_degree')
    patients2.append(patientID)
    patients3.append(patient)
    print(patient, patientID, len(slices), slices[0].pixel_array.shape)

C:\Users\casti\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


LIDC-IDRI-0001 LIDC-IDRI-0003 140 (512, 512)
LIDC-IDRI-0002 LIDC-IDRI-0005 133 (512, 512)
LIDC-IDRI-0004 LIDC-IDRI-0001 133 (512, 512)
LIDC-IDRI-0005 LIDC-IDRI-0002 261 (512, 512)


In [21]:
len(patients)

4

In [22]:
len(patients2)

4

In [23]:
len(patients3)

4

In [4]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math
%matplotlib inline

In [12]:
IMG_PX_SIZE = 50
HM_SLICES = 20

slices=[]

def chunks(l, n):
    #yield succesive n-sized chunks from l. source: Ned Batchelder.
    for i in range(0, len(l), n):
        yield l[i:i+n]
        
def mean(l):
    return sum(l)/len(l)

def process_data(patient, labels_df, img_px_size=50, hm_slices=20, visualize=False):
    
    path = data_dir + patient
    fileammount=len(os.listdir(path))
    if fileammount<=10:
        pass
    
    else:
        path = data_dir + patient
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        patientID = slices[0].PatientID
        label = labels_df.get_value(patientID, 'severity_degree')
    
    new_slices = []
    
    slices2 = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE, IMG_PX_SIZE)) for each_slice in slices]
    
    chunk_sizes = math.ceil(len(slices2) / HM_SLICES)
    
    for slice_chunk in chunks(slices2, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == HM_SLICES-1:
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
    
    if len(new_slices) == HM_SLICES-3:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES-4:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES-5:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES+2:
        new_val=list(map(mean), zip(*[new_slices[HM_SLICES-1], new_slices[HM_SLICES]]))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1]=new_val
        
    if len(new_slices) == HM_SLICES+2:
        new_val=list(map(mean), zip(*[new_slices[HM_SLICES-1], new_slices[HM_SLICES]]))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1]=new_val        

    if visualize:
        fig = plt.figure(figsize=(20,20))
        for num,each_slice in enumerate(new_slices[:1]):
            y = fig.add_subplot(4, 5, num + 1)
            y.imshow(each_slice)
        plt.show()
    
    if label == 1:
        label = np.array([0,1])
    elif label == 0:
        label = np.array([1,0])
        
    return np.array(new_slices), label

much_data = []

for num, patient in enumerate(patients3):
    if num%4==0:
        print(num)
    
    try:
        img_data, label = process_data(patient, labels_df, img_px_size=IMG_PX_SIZE, hm_slices=HM_SLICES)
        much_data.append([img_data, label])
    except KeyError as e:
        print('this is unlabeled data')

np.save('muchdataPatients-{}-{}-{}.npy'.format(IMG_PX_SIZE, IMG_PX_SIZE, HM_SLICES), much_data)

0


C:\Users\casti\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


4
8
12
16
20
24
28
32
36
40
44


In [13]:
len(much_data)

45

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.keras.models import Model

In [8]:
import numpy as np

In [18]:
IMG_SIZE_PX=IMG_PX_SIZE

In [19]:
SLICE_COUNT=HM_SLICES

In [20]:
n_classes=2

In [21]:
x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.9
keep_prob = tf.placeholder(tf.float32)

model_path = "C:/Users/casti/Documents/Final_Project/tmp/modeltestpatients.ckpt"

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               'W_fc':tf.Variable(tf.random_normal([54080,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']
    print(fc)
    return output

def train_neural_network(x):
    
    much_data = np.load('muchdataPatients-{}-{}-{}.npy'.format(IMG_PX_SIZE, IMG_PX_SIZE, HM_SLICES), allow_pickle=True)
    train_data, validation_data = train_test_split(much_data, random_state=42, test_size=0.25)
    
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    hm_epochs = 2
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(hm_epochs):
            epoch_loss = 0
            success_total=0
            attemp_total=0
            for data in train_data:
                attemp_total +=1
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    success_total = success_total+1
                except Exception as e:
                    pass
                 
        
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss, 'success rate:', success_total/attemp_total)
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
            if epoch == (hm_epochs-1):
                save_path = saver.save(sess, model_path)
                print("Model saved in file: %s" % save_path)
                
 #       correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
 #       accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
 #       print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
    

train_neural_network(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("dropout/mul:0", shape=(?, 1024), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 1 completed out of 2 loss: 43310694400.0 success rate: 0.7878787878787878


ValueError: setting an array element with a sequence.